### Whisp a feature collection

Import packages
- NB to avoid complications from altering main python environment run using a virtual environment (https://docs.python.org/3/tutorial/venv.html)

In [ ]:
import ee

import pandas as pd

from pathlib import Path

!pip install -i https://test.pypi.org/simple/ whisp

import whisp

ee.Authenticate()
ee.Initialize() 

Get a feature collection

In [ ]:
GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")
print (GEOJSON_EXAMPLE_FILEPATH)

Whisp it

In [ ]:
df_formatted_stats = whisp.whisp_formatted_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH)

Display table

In [ ]:
df_formatted_stats

Calculate risk category

In [ ]:
# add risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(df=df_formatted_stats)

Display table with risk columns

In [23]:
df_w_risk

,plotId,external_id,Area,Geometry_type,Country,ProducerCountry,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,...,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,geo,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,1,NaN,1.939000,Polygon,GHA,GH,Ashanti Region,-1.611942,6.159540,ha,...,0.000,0.000,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[-1.61283...",yes,yes,yes,no,low
1,2,NaN,4.152000,Polygon,GHA,GH,Ashanti Region,-1.644732,6.104735,ha,...,0.089,0.743,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[-1.64615...",yes,no,yes,yes,low
2,3,NaN,16.600000,Polygon,GHA,GH,Western Region,-2.157144,5.981149,ha,...,0.000,0.000,0.000,1.203,"{'type': 'Polygon', 'coordinates': [[[-2.15951...",yes,no,yes,yes,low
3,4,NaN,31.212999,Polygon,IDN,ID,South Sumatra,103.956096,-3.054668,ha,...,0.000,0.000,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[103.9514...",yes,yes,yes,no,low
4,5,NaN,1.964000,Polygon,IDN,ID,South Sumatra,103.970371,-3.068831,ha,...,0.000,0.000,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[103.9694...",yes,yes,yes,no,low
5,6,NaN,12.725000,Polygon,IDN,ID,South Sumatra,103.975182,-3.082922,ha,...,0.000,0.000,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[103.9731...",yes,no,yes,no,low
6,7,NaN,20.882000,Polygon,IDN,ID,South Sumatra,103.977512,-3.083808,ha,...,0.000,0.000,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[103.9749...",yes,yes,yes,yes,low
7,8,NaN,8.279000,Polygon,CIV,CI,Lagunes,-4.101646,5.711935,ha,...,0.000,1.602,0.000,0.001,"{'type': 'Polygon', 'coordinates': [[[-4.10288...",yes,yes,yes,yes,low
8,9,NaN,1.981000,Polygon,CIV,CI,Lagunes,-4.086848,5.673811,ha,...,0.000,0.000,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[-4.08767...",yes,yes,yes,no,low
9,10,NaN,3.797000,Polygon,CIV,CI,District Autonome D'Abidjan,-4.119589,5.572136,ha,...,0.000,0.000,0.000,0.000,"{'type': 'Polygon', 'coordinates': [[[-4.12062...",yes,yes,yes,no,low


Export table to CSV

In [24]:

# Define the output folder
out_directory = Path.home() / 'Downloads'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table_w_risk.csv'

# Save the CSV file
df_w_risk.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Table with risk columns saved to: C:\Users\Arnell\Downloads\whisp_output_table_w_risk.csv
GeoJSON saved to C:\Users\Arnell\Downloads\whisp_output_table.geojson
GeoJSON file saved to: C:\Users\Arnell\Downloads\whisp_output_table.geojson


Export to GeoJSON (optional)

In [ ]:
# Define the output file path for GeoJSON
geojson_output_file = out_directory / 'whisp_output_table.geojson'

# Save the GeoJSON file
whisp.df_to_geojson(df_w_risk, geojson_output_file)  # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.
print(f"GeoJSON file saved to: {geojson_output_file}")